In [10]:
# 导入 pyope 核心模块
from pyope import (
    BasisOperator,           # 基本算符
    DerivativeOperator,      # 导数算符
    NormalOrderedOperator,   # 正规序算符
    d, dn,                   # 导数函数
    One, Zero, Delta,        # 常数算符
    is_local_operator,       # 判断函数

)

# 导入 sympy 用于符号计算
import sympy as sp
from sympy import symbols, simplify

# 启用 LaTeX 渲染（在 Jupyter 中显示数学公式）
sp.init_printing(use_latex='mathjax')

print("PyOPE 模块导入成功！")

PyOPE 模块导入成功！


## 9. 新实现的核心功能：OPE 计算

以下是刚刚实现的核心 OPE 计算功能的演示。

### 9.1 导入 OPE 相关模块

In [11]:
# 导入 OPE 计算相关的新功能
from pyope import (
    OPE,           # OPE 计算器（支持定义和计算）
    NO,            # 正规序函数
    bracket,       # bracket 函数
    MakeOPE,       # 创建 OPEData
    Bosonic,       # 声明玻色算符
    Fermionic,     # 声明费米算符
    OPEData,       # OPE 数据类
)

print("✓ OPE 计算模块导入成功！")

✓ OPE 计算模块导入成功！


### 9.2 声明算符类型

在定义 OPE 之前，我们需要声明算符是玻色子还是费米子。

In [12]:
β = BasisOperator(r"beta", bosonic=True, conformal_weight=3/2)
γ = BasisOperator(r"gamma", bosonic=True, conformal_weight=-1/2)
b = BasisOperator("b", bosonic=False, conformal_weight=2)
c = BasisOperator("c", bosonic=False, conformal_weight=-1)

In [15]:
OPE[b, c] = OPE.make([One])
OPE[β, γ] = OPE.make([-1 * One])

In [19]:
OPE(2 * NO(b, c) + 3 * NO(β, γ), 2 * NO(b,c) + 3 * NO(β, γ))

OPEData({2: -5*One})

In [4]:
# 创建新的算符用于 OPE 演示
T_ope = BasisOperator("T", bosonic=True, conformal_weight=2)
J = BasisOperator("J", bosonic=True, conformal_weight=1)
psi_ope = BasisOperator("psi", bosonic=False, conformal_weight=1.5)

# 声明算符类型（注册到全局注册表）
Bosonic(T_ope, J)
Fermionic(psi_ope)

print("✓ 算符类型声明完成")
print(f"  - T_ope: 玻色算符")
print(f"  - J: 玻色算符")
print(f"  - psi_ope: 费米算符")

✓ 算符类型声明完成
  - T_ope: 玻色算符
  - J: 玻色算符
  - psi_ope: 费米算符


### 9.3 定义 OPE

使用 `OPE[A, B] = ...` 语法定义基本算符的 OPE。

In [5]:
# 定义 Virasoro 代数的 OPE: T(z)T(w)
# T(z)T(w) = (c/2)/(z-w)^4 + 2T(w)/(z-w)^2 + ∂T(w)/(z-w)

c = sp.Symbol('c')  # 中心荷

# 使用 MakeOPE 或 OPE.make 创建 OPEData
# 列表按从高阶到低阶排列：[pole_4, pole_3, pole_2, pole_1]
OPE[T_ope, T_ope] = OPE.make([
    c/2 * One,    # 4阶极点
    0,            # 3阶极点（无）
    2 * T_ope,    # 2阶极点
    d(T_ope)      # 1阶极点
])

print("✓ 定义了 Virasoro 代数的 OPE:")
print(f"  OPE[T, T] = (c/2)/(z-w)^4 + 2T/(z-w)^2 + ∂T/(z-w)")

✓ 定义了 Virasoro 代数的 OPE:
  OPE[T, T] = (c/2)/(z-w)^4 + 2T/(z-w)^2 + ∂T/(z-w)


In [7]:
OPE(T_ope,T_ope)

OPEData({4: One*c/2, 2: 2*T, 1: ∂T})

In [47]:
# 定义更简单的 OPE: J(z)J(w)
# J(z)J(w) = k/(z-w)^2

k = sp.Symbol('k')  # 水平

OPE[J, J] = OPE.make([
    k * One,      # 2阶极点
    0             # 1阶极点（无）
])

print("✓ 定义了电流代数的 OPE:")
print(f"  OPE[J, J] = k/(z-w)^2")

✓ 定义了电流代数的 OPE:
  OPE[J, J] = k/(z-w)^2


### 9.4 计算 OPE

使用 `OPE(A, B)` 计算两个算符的 OPE。

In [48]:
# 计算 T(z)T(w) 的 OPE
ope_TT = OPE(T_ope, T_ope)

print("计算 OPE(T, T):")
print(f"  类型: {type(ope_TT).__name__}")
print(f"  最高极点: {ope_TT.max_pole}")
print(f"  极点字典: {ope_TT.poles}")
print()

# 查看各个极点的系数
print("各阶极点的系数:")
for n in range(ope_TT.max_pole, 0, -1):
    coeff = ope_TT.pole(n)
    if coeff != 0:
        print(f"  pole({n}): {coeff}")

计算 OPE(T, T):
  类型: OPEData
  最高极点: 4
  极点字典: {4: One*c/2, 2: 2*T, 1: d(T)}

各阶极点的系数:
  pole(4): One*c/2
  pole(2): 2*T
  pole(1): ∂T


In [49]:
# 计算 J(z)J(w) 的 OPE
ope_JJ = OPE(J, J)

print("计算 OPE(J, J):")
print(f"  最高极点: {ope_JJ.max_pole}")
print(f"  pole(2): {ope_JJ.pole(2)}")
print(f"  pole(1): {ope_JJ.pole(1)}")

计算 OPE(J, J):
  最高极点: 2
  pole(2): One*k
  pole(1): 0


### 9.5 OPE 的线性性

OPE 计算自动支持线性性：`OPE(A, B+C) = OPE(A,B) + OPE(A,C)`

In [50]:
# 测试线性性：OPE(T, J+J) = OPE(T,J) + OPE(T,J)
# 注意：T 和 J 的 OPE 未定义，所以返回零

ope_sum = OPE(T_ope, J + J)
print(f"OPE(T, J+J) = {ope_sum}")
print(f"是否为零: {ope_sum.is_zero()}")
print()

# 测试标量乘法：OPE(2*T, T) = 2*OPE(T,T)
ope_scaled = OPE(2*T_ope, T_ope)
print(f"OPE(2*T, T) 的最高极点: {ope_scaled.max_pole}")
print(f"pole(4) = {ope_scaled.pole(4)}")  # 应该是 2*(c/2) = c
print(f"pole(2) = {ope_scaled.pole(2)}")  # 应该是 2*2*T = 4*T

OPE(T, J+J) = (2*J)/(z-w)^2 + (2*∂J)/(z-w)
是否为零: False

OPE(2*T, T) 的最高极点: 4
pole(4) = One*c
pole(2) = 4*T


### 9.6 使用 bracket 函数

`bracket(A, B, n)` 提取 OPE 的第 n 阶极点系数，即 {AB}_n。

In [51]:
# 使用 bracket 提取特定极点
bracket_4 = bracket(T_ope, T_ope, 4)
bracket_2 = bracket(T_ope, T_ope, 2)
bracket_1 = bracket(T_ope, T_ope, 1)
bracket_0 = bracket(T_ope, T_ope, 0)

print("使用 bracket 函数提取 OPE 的极点系数:")
print(f"  {{TT}}_4 = {bracket_4}")
print(f"  {{TT}}_2 = {bracket_2}")
print(f"  {{TT}}_1 = {bracket_1}")
print(f"  {{TT}}_0 = {bracket_0}")
print()
print("注意：bracket(A, B, 0) = NO(A, B) 是正规序乘积")

使用 bracket 函数提取 OPE 的极点系数:
  {TT}_4 = One*c/2
  {TT}_2 = 2*T
  {TT}_1 = ∂T
  {TT}_0 = 0

注意：bracket(A, B, 0) = NO(A, B) 是正规序乘积


### 9.7 使用 NO 函数

`NO(A, B)` 计算正规序乘积，等价于 `bracket(A, B, 0)`。

In [52]:
# 使用 NO 函数创建正规序乘积
no_TJ = NO(T_ope, J)
print(f"NO(T, J) = {no_TJ}")
print(f"类型: {type(no_TJ).__name__}")
print()

# NO 函数支持线性性
no_sum = NO(T_ope + J, J)
print(f"NO(T+J, J) = {no_sum}")
print()

# NO 函数支持标量乘法
no_scaled = NO(2*T_ope, J)
print(f"NO(2*T, J) = {no_scaled}")
print()

# NO 与零算符
no_zero = NO(T_ope, 0)
print(f"NO(T, 0) = {no_zero}")

NO(T, J) = NO(T,J)
类型: NormalOrderedOperator

NO(T+J, J) = NO(J,J) + NO(T,J)

NO(2*T, J) = 2*NO(T,J)

NO(T, 0) = 0


### 9.8 对易子和反对易子

`bracket` 函数还可以计算对易子和反对易子。

In [53]:
# 玻色算符的对易子: [A, B] = NO(A,B) - NO(B,A)
commutator_TJ = bracket(T_ope, J, anticommutator=False)
print(f"对易子 [T, J] = {commutator_TJ}")
print()

# 费米算符的反对易子: {A, B} = NO(A,B) + NO(B,A)
anticommutator_psi = bracket(psi_ope, psi_ope, anticommutator=True)
print(f"反对易子 {{psi, psi}} = {anticommutator_psi}")

对易子 [T, J] = -NO(J,T) + NO(T,J)

反对易子 {psi, psi} = 2*NO(psi,psi)


### 9.9 OPEData 的算术运算

OPEData 对象支持加法、减法和标量乘法。

In [54]:
# OPEData 的加法
ope1 = OPE(T_ope, T_ope)
ope2 = OPE(J, J)
ope_sum = ope1 + ope2

print("OPEData 加法:")
print(f"  OPE(T,T) + OPE(J,J) 的最高极点: {ope_sum.max_pole}")
print(f"  pole(4): {ope_sum.pole(4)}")
print(f"  pole(2): {ope_sum.pole(2)}")
print()

# OPEData 的标量乘法
ope_scaled = 3 * ope1
print("OPEData 标量乘法:")
print(f"  3 * OPE(T,T) 的 pole(4): {ope_scaled.pole(4)}")
print(f"  3 * OPE(T,T) 的 pole(2): {ope_scaled.pole(2)}")
print()

# OPEData 的减法
ope_diff = ope1 - ope1
print("OPEData 减法:")
print(f"  OPE(T,T) - OPE(T,T) 是否为零: {ope_diff.is_zero()}")

OPEData 加法:
  OPE(T,T) + OPE(J,J) 的最高极点: 4
  pole(4): One*c/2
  pole(2): 2*T + One*k

OPEData 标量乘法:
  3 * OPE(T,T) 的 pole(4): 3*One*c/2
  3 * OPE(T,T) 的 pole(2): 6*T

OPEData 减法:
  OPE(T,T) - OPE(T,T) 是否为零: True


### 9.10 复合算符的 OPE（完整实现）

PyOPE 现在完整支持复合算符的 OPE 计算，包括：
- 右侧复合：`OPE(A, NO(B,C))`
- 左侧复合：`OPE(NO(A,B), C)` ✨ **新功能**

这些计算使用完整的 Jacobi 恒等式实现。

In [55]:
OPE[T_ope, J] = OPE.make([J, d(J)])

In [56]:
OPE(T_ope, d(d(J)))

OPEData({4: 6*J, 3: 6*∂J, 2: 3*∂^2J, 1: ∂^3J})

In [57]:
OPE(d(T_ope), J)

OPEData({3: -2*J, 2: -∂J})

In [58]:
# 创建嵌套的正规序算符
no_JJ = NO(J, J)
no_nested = NO(no_JJ, J)

print(f"嵌套正规序: {no_nested}")
print(f"类型: {type(no_nested).__name__}")
print()

# 计算嵌套正规序算符的 OPE
ope_nested = OPE(no_nested, J)
print(f"OPE(NO(NO(J,J), J), J) 最高极点: {ope_nested.max_pole}")
print()
print("各阶极点:")
for q in range(min(ope_nested.max_pole, 3), 0, -1):
    pole_q = ope_nested.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

嵌套正规序: NO(NO(J,J),J)
类型: NormalOrderedOperator

OPE(NO(NO(J,J), J), J) 最高极点: 4

各阶极点:
  pole(2): 3*NO(J,J)*k
  pole(1): ∂NO(J,J)*k + 2*NO(J,∂J)*k + 2*NO(∂J,J)*k


### 9.11 嵌套正规序算符

PyOPE 支持嵌套的正规序算符，例如 `NO(NO(A,B), C)`。

In [59]:
# 示例 3: 比较左侧和右侧复合算符的 OPE
# OPE(NO(J,J), J) vs OPE(J, NO(J,J))

no_JJ = NO(J, J)
ope_left = OPE(no_JJ, J)
ope_right = OPE(J, no_JJ)

print("示例 3: 左侧 vs 右侧复合算符")
print(f"OPE(NO(J,J), J) 最高极点: {ope_left.max_pole}")
print(f"OPE(J, NO(J,J)) 最高极点: {ope_right.max_pole}")
print()
print("左侧复合 - pole(2):", ope_left.pole(2))
print("右侧复合 - pole(2):", ope_right.pole(2))

示例 3: 左侧 vs 右侧复合算符
OPE(NO(J,J), J) 最高极点: 2
OPE(J, NO(J,J)) 最高极点: 2

左侧复合 - pole(2): 2*J*k
右侧复合 - pole(2): 2*J*k


In [60]:
# 示例 2: 计算 OPE(NO(T,J), J)
# 混合算符的左侧复合 OPE

no_TJ = NO(T_ope, J)
ope_mixed = OPE(no_TJ, J)

print("示例 2: OPE(NO(T,J), J)")
print(f"最高极点: {ope_mixed.max_pole}")
print()
print("各阶极点:")
for q in range(ope_mixed.max_pole, 0, -1):
    pole_q = ope_mixed.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")
        
ope_mixed.pole(1)

示例 2: OPE(NO(T,J), J)
最高极点: 4

各阶极点:
  pole(4): 3*One*k
  pole(2): T*k + NO(J,J)
  pole(1): ∂T*k + NO(J,∂J) + NO(∂J,J)


∂T⋅k + NO(J,∂J) + NO(∂J,J)

In [61]:
# 示例 1: 计算 OPE(NO(J,J), J)
# 这是左侧复合算符的 OPE

no_JJ = NO(J, J)
ope_left = OPE(no_JJ, J)

print("示例 1: OPE(NO(J,J), J)")
print(f"最高极点: {ope_left.max_pole}")
print()
print("各阶极点:")
for q in range(ope_left.max_pole, 0, -1):
    pole_q = ope_left.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

示例 1: OPE(NO(J,J), J)
最高极点: 2

各阶极点:
  pole(2): 2*J*k
  pole(1): 2*∂J*k


In [62]:
# 定义 T 和 J 的混合 OPE
OPE[T_ope, J] = OPE.make([
    J,          # pole 2
    d(J)        # pole 1
])

print("✓ 定义了混合 OPE: T(z)J(w) = J/(z-w)^2 + ∂J/(z-w)")

✓ 定义了混合 OPE: T(z)J(w) = J/(z-w)^2 + ∂J/(z-w)


## 10. 高级功能：复杂的 OPE 计算

以下演示更加复杂和有趣的 OPE 计算功能。

### 10.1 多重导数算符的 OPE

PyOPE 支持高阶导数算符的 OPE 计算，通过 Leibniz 规则自动展开。

In [63]:
# 示例 1: 二阶导数与基本算符的 OPE
# OPE(∂²T, J) 应该通过导数规则计算

ope_d2T_J = OPE(d(d(T_ope)), J)

print("示例 1: OPE(∂²T, J)")
print(f"最高极点: {ope_d2T_J.max_pole}")
print()
print("各阶极点:")
for q in range(ope_d2T_J.max_pole, 0, -1):
    pole_q = ope_d2T_J.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

示例 1: OPE(∂²T, J)
最高极点: 4

各阶极点:
  pole(4): 6*J
  pole(3): 2*∂J


In [64]:
ope_d2T_J = OPE(d(d(T_ope)), J)

print("示例 1: OPE(∂²T, J)")
print(f"最高极点: {ope_d2T_J.max_pole}")
print()
print("各阶极点:")
for q in range(ope_d2T_J.max_pole, 0, -1):
    pole_q = ope_d2T_J.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

示例 1: OPE(∂²T, J)
最高极点: 4

各阶极点:
  pole(4): 6*J
  pole(3): 2*∂J


In [ ]:
# 示例 2: 三阶导数的 OPE
# OPE(T, ∂³J) - 右侧高阶导数

ope_T_d3J = OPE(T_ope, dn(3, J))

print("示例 2: OPE(T, ∂³J)")
print(f"最高极点: {ope_T_d3J.max_pole}")
print()
print("各阶极点 (仅显示前 5 个):")
for q in range(min(ope_T_d3J.max_pole, 5), 0, -1):
    pole_q = ope_T_d3J.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

In [ ]:
# 示例 3: 双侧导数算符的 OPE
# OPE(∂T, ∂J)

ope_dT_dJ = OPE(d(T_ope), d(J))

print("示例 3: OPE(∂T, ∂J)")
print(f"最高极点: {ope_dT_dJ.max_pole}")
print()
print("各阶极点:")
for q in range(ope_dT_dJ.max_pole, 0, -1):
    pole_q = ope_dT_dJ.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

### 10.2 费米算符系统的完整 OPE

展示费米算符的 OPE 计算，包括反对易关系和超对称代数。

In [ ]:
# 定义费米算符系统: 自由费米子
# 定义两个费米算符 psi 和 chi

chi = BasisOperator("chi", bosonic=False, conformal_weight=0.5)
Fermionic(chi)

# 定义费米算符的 OPE
# psi(z)psi(w) ~ 1/(z-w)
# chi(z)chi(w) ~ 1/(z-w)
# psi(z)chi(w) ~ 0 (不同费米子无相互作用)

OPE[psi_ope, psi_ope] = OPE.make([One])
OPE[chi, chi] = OPE.make([One])

print("✓ 定义了费米算符系统:")
print(f"  OPE[psi, psi] = 1/(z-w)")
print(f"  OPE[chi, chi] = 1/(z-w)")

In [21]:

chi = BasisOperator("chi", bosonic=False, conformal_weight=0.5)
Fermionic(chi)

In [22]:

# 定义费米算符的 OPE
# psi(z)psi(w) ~ 1/(z-w)
# chi(z)chi(w) ~ 1/(z-w)
# psi(z)chi(w) ~ 0 (不同费米子无相互作用)
OPE[psi_ope, psi_ope] = OPE.make([One])
OPE[chi, chi] = OPE.make([One])

print("✓ 定义了费米算符系统:")
print(f"  OPE[psi, psi] = 1/(z-w)")
print(f"  OPE[chi, chi] = 1/(z-w)")

✓ 定义了费米算符系统:
  OPE[psi, psi] = 1/(z-w)
  OPE[chi, chi] = 1/(z-w)


In [30]:
# 计算费米算符的反对易子
# {psi, psi} = 2*NO(psi, psi)

anticomm_psi = bracket(psi_ope, psi_ope, anticommutator=True)
print(f"反对易子 {{psi, psi}} = {anticomm_psi}")
print(f"类型: {type(anticomm_psi).__name__}")
print()

# 计算不同费米算符的反对易子
# {psi, chi} = 2*NO(psi, chi) (因为它们的 OPE 未定义)

anticomm_psi_chi = bracket(psi_ope, chi, anticommutator=True)
print(f"反对易子 {{psi, chi}} = {anticomm_psi_chi}")
print(f"说明: 不同费米子无相互作用")

反对易子 {psi, psi} = 2*NO(psi,psi)
类型: Mul

反对易子 {psi, chi} = NO(chi,psi) + NO(psi,chi)
说明: 不同费米子无相互作用


In [ ]:
# 定义超对称流算符 (supercurrent)
# G = NO(psi, ∂chi) (简化版本)

G = NO(psi_ope, d(chi))
print(f"超对称流: G = {G}")
print(f"共形权重: {G.conformal_weight}")
print()

# 计算 G 与费米算符的 OPE
ope_G_psi = OPE(G, psi_ope)
print(f"OPE(G, psi) 最高极点: {ope_G_psi.max_pole}")
if not ope_G_psi.is_zero():
    print("各阶极点:")
    for q in range(ope_G_psi.max_pole, 0, -1):
        pole_q = ope_G_psi.pole(q)
        if pole_q != 0:
            print(f"  pole({q}): {pole_q}")

### 10.3 Sugawara 构造的详细计算

Sugawara 构造是从流代数 (current algebra) 构造 Virasoro 代数的标准方法。

In [ ]:
# 定义 U(1) current 的 Sugawara 张量
# T_sugawara = (1/2k) * NO(J, J)
# 其中 k 是水平 (level)

# 使用之前定义的 J 和 k
no_JJ = NO(J, J)
T_sugawara = no_JJ / (2 * k)

print(f"Sugawara 张量: T_sugawara = (1/2k) * NO(J, J)")
print(f"展开形式: T_sugawara = {T_sugawara}")
print(f"基础算符 NO(J,J) 的共形权重: {no_JJ.conformal_weight}")
print(f"说明: T_sugawara 是标量乘积，其共形权重与 NO(J,J) 相同")
print()

# 计算 T_sugawara 与 J 的 OPE
# 期望结果: T(z)J(w) ~ J/(z-w)^2 + ∂J/(z-w)

ope_Tsug_J = OPE(T_sugawara, J)
print("OPE(T_sugawara, J):")
print(f"最高极点: {ope_Tsug_J.max_pole}")
print()
print("各阶极点:")
for q in range(ope_Tsug_J.max_pole, 0, -1):
    pole_q = ope_Tsug_J.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

In [ ]:
# 计算 T_sugawara 的自 OPE
# 期望: 验证它满足 Virasoro 代数
# OPE(T_sugawara, T_sugawara) ~ c/(2(z-w)^4) + 2T_sugawara/(z-w)^2 + ∂T_sugawara/(z-w)
# 其中中心荷 c = 1 (对于单个 U(1) current)

ope_Tsug_Tsug = OPE(T_sugawara, T_sugawara)

print("OPE(T_sugawara, T_sugawara):")
print(f"最高极点: {ope_Tsug_Tsug.max_pole}")
print()
print("各阶极点:")
for q in range(ope_Tsug_Tsug.max_pole, 0, -1):
    pole_q = ope_Tsug_Tsug.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")
print()

# 提取中心荷
if ope_Tsug_Tsug.max_pole >= 4:
    central_charge_term = ope_Tsug_Tsug.pole(4)
    print(f"中心荷项: {central_charge_term}")
    print(f"说明: 对于 U(1) current，期望中心荷 c = 1，即 c/2 = 1/2")

### 10.4 Jacobi 恒等式的验证

Jacobi 恒等式是 VOA 的核心性质，确保 OPE 的结合性。我们可以通过计算来验证。

In [ ]:
# Jacobi 恒等式的一个例子:
# 对于三个算符 A, B, C，验证结合性
# 这里我们用 J, J, J 来验证

# 计算 OPE(J, OPE(J, J))
ope_JJ = OPE(J, J)
print("步骤 1: 计算 OPE(J, J)")
print(f"  结果: 最高极点 = {ope_JJ.max_pole}")
print(f"  pole(2) = {ope_JJ.pole(2)}")
print()

# 提取极点系数并计算 OPE(J, pole(2))
pole_2_coeff = ope_JJ.pole(2)
if pole_2_coeff != 0:
    ope_J_pole2 = OPE(J, pole_2_coeff)
    print("步骤 2: 计算 OPE(J, pole(2) of OPE(J,J))")
    print(f"  OPE(J, {pole_2_coeff}) = {ope_J_pole2}")
    print()

# 计算 OPE(OPE(J, J), J) - 左侧复合
ope_left = OPE(NO(J, J), J)
print("步骤 3: 计算 OPE(NO(J, J), J) - 左侧复合算符")
print(f"  最高极点 = {ope_left.max_pole}")
print(f"  pole(2) = {ope_left.pole(2)}")
print()

print("说明: Jacobi 恒等式验证需要比较不同顺序的 OPE 计算结果")

In [ ]:
# Virasoro 代数的 Jacobi 恒等式验证
# 验证: [L_m, [L_n, L_p]] + 循环置换 = 0
# 这里我们通过 OPE 计算来检验部分性质

# 计算 {TT}_n 与 T 的 OPE，验证结合律
bracket_TT_2 = bracket(T_ope, T_ope, 2)  # {TT}_2 = 2T
print(f"提取 bracket: {{TT}}_2 = {bracket_TT_2}")
print()

# 计算 OPE({TT}_2, T)
ope_bracket_T = OPE(bracket_TT_2, T_ope)
print(f"OPE({{TT}}_2, T) 最高极点: {ope_bracket_T.max_pole}")
print()

# 比较: OPE(2T, T) = 2 * OPE(T, T)
ope_2T_T = 2 * OPE(T_ope, T_ope)
print(f"2 * OPE(T, T) 最高极点: {ope_2T_T.max_pole}")
print()

# 验证线性性
print("验证线性性: OPE({TT}_2, T) 应该等于 2 * OPE(T, T)")
print(f"  pole(4) 比较: {ope_bracket_T.pole(4)} vs {ope_2T_T.pole(4)}")
print(f"  pole(2) 比较: {ope_bracket_T.pole(2)} vs {ope_2T_T.pole(2)}")
print(f"  相等? {(ope_bracket_T.pole(4) == ope_2T_T.pole(4)) and (ope_bracket_T.pole(2) == ope_2T_T.pole(2))}")

### 10.5 多算符连续 OPE 计算

展示如何计算涉及多个算符的连续 OPE。

In [ ]:
# 示例 1: 三重正规序算符
# NO(NO(J, J), NO(J, J))

no_JJ = NO(J, J)
no_triple = NO(no_JJ, no_JJ)

print(f"三重复合: {no_triple}")
print(f"共形权重: {no_triple.conformal_weight}")
print()

# 计算与基本算符的 OPE
ope_triple = OPE(no_triple, J)
print(f"OPE(NO(NO(J,J), NO(J,J)), J):")
print(f"  最高极点: {ope_triple.max_pole}")
if ope_triple.max_pole > 0:
    print("  前 3 个极点:")
    for q in range(min(ope_triple.max_pole, 3), 0, -1):
        pole_q = ope_triple.pole(q)
        if pole_q != 0:
            print(f"    pole({q}): {pole_q}")

In [ ]:
# 示例 2: 混合算符的连续 OPE
# 计算 OPE(T, NO(T, J))

no_TJ = NO(T_ope, J)
ope_T_TJ = OPE(T_ope, no_TJ)

print(f"OPE(T, NO(T, J)):")
print(f"  最高极点: {ope_T_TJ.max_pole}")
print()
if ope_T_TJ.max_pole > 0:
    print("  各阶极点 (显示前 4 个):")
    for q in range(min(ope_T_TJ.max_pole, 4), 0, -1):
        pole_q = ope_T_TJ.pole(q)
        if pole_q != 0:
            print(f"    pole({q}): {pole_q}")

In [ ]:
# 示例 3: 导数算符与复合算符的 OPE
# 计算 OPE(∂T, NO(J, J))

ope_dT_JJ = OPE(d(T_ope), no_JJ)

print(f"OPE(∂T, NO(J, J)):")
print(f"  最高极点: {ope_dT_JJ.max_pole}")
print()
if ope_dT_JJ.max_pole > 0:
    print("  各阶极点 (显示前 3 个):")
    for q in range(min(ope_dT_JJ.max_pole, 3), 0, -1):
        pole_q = ope_dT_JJ.pole(q)
        if pole_q != 0:
            print(f"    pole({q}): {pole_q}")

### 10.6 高阶极点的处理

展示如何处理高阶极点的 OPE 计算。

In [ ]:
# 定义一个高共形权重的算符
W = BasisOperator("W", bosonic=True, conformal_weight=3)
Bosonic(W)

# 定义它与 T 的 OPE (假设的 W_3 代数)
# W(z)T(w) ~ 3W/(z-w)^2 + ∂W/(z-w)
OPE[W, T_ope] = OPE.make([
    3 * W,      # pole 2
    d(W)        # pole 1
])

# 定义 W 的自 OPE (简化版本)
# W(z)W(w) ~ c_W/(z-w)^6 + ... + W/(z-w)^2
c_W = sp.Symbol('c_W')
OPE[W, W] = OPE.make([
    c_W * One,  # pole 6
    0,          # pole 5
    0,          # pole 4
    0,          # pole 3
    W,          # pole 2
    d(W)        # pole 1
])

print("✓ 定义了高共形权重算符 W (权重 = 3)")
print(f"  OPE[W, T] = 3W/(z-w)^2 + ∂W/(z-w)")
print(f"  OPE[W, W] 有 6 阶极点")

In [ ]:
# 计算 W 的自 OPE
ope_WW = OPE(W, W)

print("OPE(W, W):")
print(f"  最高极点: {ope_WW.max_pole}")
print()
print("  所有非零极点:")
for q in range(ope_WW.max_pole, 0, -1):
    pole_q = ope_WW.pole(q)
    if pole_q != 0:
        print(f"    pole({q}): {pole_q}")

In [ ]:
# 计算 T 与 W 的 OPE
ope_TW = OPE(T_ope, W)

print("OPE(T, W):")
print(f"  最高极点: {ope_TW.max_pole}")
print()
print("  各阶极点:")
for q in range(ope_TW.max_pole, 0, -1):
    pole_q = ope_TW.pole(q)
    if pole_q != 0:
        print(f"    pole({q}): {pole_q}")
print()

# 验证共形权重的变换性质
# OPE(T, φ) ~ h*φ/(z-w)^2 + ∂φ/(z-w)
# 这里 h = 3 (W 的共形权重)
print("验证共形变换性质:")
print(f"  期望 pole(2) = {W.conformal_weight} * W = {W.conformal_weight * W}")
print(f"  实际 pole(2) = {ope_TW.pole(2)}")
print(f"  匹配? {ope_TW.pole(2) == W.conformal_weight * W}")

### 10.7 符号计算与简化

展示如何使用 sympy 的符号简化功能处理 OPE 结果。

In [ ]:
# 计算一个复杂的 OPE 并简化结果
# 例如: OPE(T + 2J, T + 2J)

complex_op = T_ope + 2*J

ope_complex = OPE(complex_op, complex_op)

print("OPE(T + 2J, T + 2J):")
print(f"  最高极点: {ope_complex.max_pole}")
print()

# 使用 sympy 的 simplify 简化每个极点
print("  各阶极点 (简化后):")
for q in range(ope_complex.max_pole, 0, -1):
    pole_q = ope_complex.pole(q)
    if pole_q != 0:
        # 尝试简化（如果是 sympy 表达式）
        try:
            simplified = simplify(pole_q)
            print(f"    pole({q}): {simplified}")
        except:
            print(f"    pole({q}): {pole_q}")

In [ ]:
# 使用符号参数进行参数化计算
# 定义一个参数化的算符组合

alpha, beta = sp.symbols('alpha beta')
param_op = alpha * T_ope + beta * J

print(f"参数化算符: {alpha}*T + {beta}*J")
print()

# 计算它与 J 的 OPE
ope_param = OPE(param_op, J)

print(f"OPE({alpha}*T + {beta}*J, J):")
print(f"  最高极点: {ope_param.max_pole}")
print()
print("  各阶极点:")
for q in range(ope_param.max_pole, 0, -1):
    pole_q = ope_param.pole(q)
    if pole_q != 0:
        simplified = simplify(pole_q)
        print(f"    pole({q}): {simplified}")

## 11. 总结与展望

本 notebook 展示了 PyOPE 库的主要功能，包括：

1. **基本 OPE 计算**: 定义和计算基本算符的 OPE
2. **复合算符**: 支持正规序算符 (NO) 和嵌套结构
3. **导数规则**: 自动处理导数算符的 OPE
4. **费米算符**: 完整支持费米子系统和反对易关系
5. **Sugawara 构造**: 从流代数构造 Virasoro 代数
6. **Jacobi 恒等式**: 验证 OPE 的结合性
7. **高阶极点**: 处理高共形权重算符
8. **符号计算**: 与 sympy 无缝集成

PyOPE 是一个强大的顶点算符代数符号计算工具，适用于：
- 共形场论研究
- W-代数计算
- 顶点算符代数理论
- 字符串理论中的对称性分析

**未来发展方向**:
- 更多内置的代数结构 (如 Kac-Moody 代数)
- OPEJacobi 函数的完整实现
- 性能优化和并行计算
- 更丰富的可视化功能

### 9.9 OPEData 的算术运算

OPEData 对象支持加法、减法和标量乘法。

In [12]:
# OPEData 的加法
ope1 = OPE(T_ope, T_ope)
ope2 = OPE(J, J)
ope_sum = ope1 + ope2

print("OPEData 加法:")
print(f"  OPE(T,T) + OPE(J,J) 的最高极点: {ope_sum.max_pole}")
print(f"  pole(4): {ope_sum.pole(4)}")
print(f"  pole(2): {ope_sum.pole(2)}")
print()

# OPEData 的标量乘法
ope_scaled = 3 * ope1
print("OPEData 标量乘法:")
print(f"  3 * OPE(T,T) 的 pole(4): {ope_scaled.pole(4)}")
print(f"  3 * OPE(T,T) 的 pole(2): {ope_scaled.pole(2)}")
print()

# OPEData 的减法
ope_diff = ope1 - ope1
print("OPEData 减法:")
print(f"  OPE(T,T) - OPE(T,T) 是否为零: {ope_diff.is_zero()}")

OPEData 加法:
  OPE(T,T) + OPE(J,J) 的最高极点: 4
  pole(4): One*c/2
  pole(2): 2*T + One*k

OPEData 标量乘法:
  3 * OPE(T,T) 的 pole(4): 3*One*c/2
  3 * OPE(T,T) 的 pole(2): 6*T

OPEData 减法:
  OPE(T,T) - OPE(T,T) 是否为零: True


### 9.10 复合算符的 OPE（完整实现）

PyOPE 现在完整支持复合算符的 OPE 计算，包括：
- 右侧复合：`OPE(A, NO(B,C))`
- 左侧复合：`OPE(NO(A,B), C)` ✨ **新功能**

这些计算使用完整的 Jacobi 恒等式实现。

In [13]:
OPE[T_ope, J] = OPE.make([J, d(J)])

In [14]:
OPE(T_ope, d(d(J)))

OPEData({4: 6*J, 3: 6*∂J, 2: 3*∂^2J, 1: ∂^3J})

In [15]:
OPE(d(T_ope), J)

OPEData({3: -2*J, 2: -∂J})

In [18]:
# 创建嵌套的正规序算符
no_JJ = NO(J, J)
no_nested = NO(no_JJ, J)

print(f"嵌套正规序: {no_nested}")
print(f"类型: {type(no_nested).__name__}")
print()

# 计算嵌套正规序算符的 OPE
ope_nested = OPE(no_nested, J)
print(f"OPE(NO(NO(J,J), J), J) 最高极点: {ope_nested.max_pole}")
print()
print("各阶极点:")
for q in range(min(ope_nested.max_pole, 3), 0, -1):
    pole_q = ope_nested.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

嵌套正规序: NO(NO(J,J),J)
类型: NormalOrderedOperator

OPE(NO(NO(J,J), J), J) 最高极点: 2

各阶极点:
  pole(2): 3*NO(J,J)*k
  pole(1): ∂NO(J,J)*k + 2*NO(J,∂J)*k + 2*NO(∂J,J)*k


### 9.11 嵌套正规序算符

PyOPE 支持嵌套的正规序算符，例如 `NO(NO(A,B), C)`。

In [ ]:
# 示例 3: 比较左侧和右侧复合算符的 OPE
# OPE(NO(J,J), J) vs OPE(J, NO(J,J))

no_JJ = NO(J, J)
ope_left = OPE(no_JJ, J)
ope_right = OPE(J, no_JJ)

print("示例 3: 左侧 vs 右侧复合算符")
print(f"OPE(NO(J,J), J) 最高极点: {ope_left.max_pole}")
print(f"OPE(J, NO(J,J)) 最高极点: {ope_right.max_pole}")
print()
print("左侧复合 - pole(2):", ope_left.pole(2))
print("右侧复合 - pole(2):", ope_right.pole(2))

In [ ]:
# 示例 2: 计算 OPE(NO(T,J), J)
# 混合算符的左侧复合 OPE

no_TJ = NO(T_ope, J)
ope_mixed = OPE(no_TJ, J)

print("示例 2: OPE(NO(T,J), J)")
print(f"最高极点: {ope_mixed.max_pole}")
print()
print("各阶极点:")
for q in range(ope_mixed.max_pole, 0, -1):
    pole_q = ope_mixed.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

In [ ]:
# 示例 1: 计算 OPE(NO(J,J), J)
# 这是左侧复合算符的 OPE

no_JJ = NO(J, J)
ope_left = OPE(no_JJ, J)

print("示例 1: OPE(NO(J,J), J)")
print(f"最高极点: {ope_left.max_pole}")
print()
print("各阶极点:")
for q in range(ope_left.max_pole, 0, -1):
    pole_q = ope_left.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

In [ ]:
# 定义 T 和 J 的混合 OPE
OPE[T_ope, J] = OPE.make([
    J,          # pole 2
    d(J)        # pole 1
])

print("✓ 定义了混合 OPE: T(z)J(w) = J/(z-w)^2 + ∂J/(z-w)")